<a href="https://colab.research.google.com/github/MarriHarshaVardhan/realtime-sentiment-analyzer/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install flask
!pip install transformers # For Hugging Face sentiment analysis model
!pip install ngrok # or serveo for tunneling
!pip install flask-ngrok
!pip install pyngrok

In [5]:
!pip show flask-ngrok

Name: flask-ngrok
Version: 0.0.25
Summary: A simple way to demo Flask apps from your machine.
Home-page: https://github.com/gstaff/flask-ngrok
Author: Grant Stafford
Author-email: 
License: UNKNOWN
Location: /usr/local/lib/python3.11/dist-packages
Requires: Flask, requests
Required-by: 


In [7]:
%%writefile app.py
from flask import Flask, render_template_string, request, jsonify
from transformers import pipeline

app = Flask(__name__)

sentiment_pipeline = pipeline("sentiment-analysis")

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Real-time Sentiment Analyzer</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 20px; }
        .container { max-width: 600px; margin: auto; padding: 20px; border: 1px solid #ccc; border-radius: 8px; }
        textarea { width: 100%; height: 150px; margin-bottom: 10px; padding: 10px; border: 1px solid #ddd; border-radius: 4px; }
        button { padding: 10px 15px; background-color: #007bff; color: white; border: none; border-radius: 4px; cursor: pointer; }
        button:hover { background-color: #0056b3; }
        #result { margin-top: 20px; font-weight: bold; }
        .positive { color: green; }
        .negative { color: red; }
        .neutral { color: gray; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Real-time Sentiment Analyzer</h1>
        <p>Type some text below and get its sentiment analyzed instantly!</p>
        <textarea id="textInput" placeholder="Enter your text here..."></textarea>
        <button onclick="analyzeSentiment()">Analyze Sentiment</button>
        <div id="result"></div>
    </div>

    <script>
        async function analyzeSentiment() {
            const text = document.getElementById('textInput').value;
            if (!text.trim()) {
                document.getElementById('result').innerHTML = '';
                return;
            }

            const response = await fetch('/analyze', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ text: text })
            });
            const data = await response.json();
            const sentiment = data.sentiment[0];
            const label = sentiment.label;
            const score = sentiment.score;

            let resultDiv = document.getElementById('result');
            resultDiv.innerHTML = `Sentiment: <span class="${label.toLowerCase()}">${label}</span> (Score: ${score.toFixed(4)})`;
        }
        document.getElementById('textInput').addEventListener('keyup', analyzeSentiment);
    </script>
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.get_json()
    text = data.get('text', '')
    if text:
        sentiment = sentiment_pipeline(text)
        return jsonify({"sentiment": sentiment})
    return jsonify({"sentiment": []}), 400

if __name__ == '__main__':
    app.run()

Overwriting app.py


In [10]:
# First, get your ngrok authtoken from https://ngrok.com/signup and paste it here
# You only need to run this once per Colab session or if your token changes.
!ngrok authtoken 2tn9hLRFrAqqE1ZCUddIa4teUZ7_7CsDvidB6bJYhJDQC4tQG

import threading
import subprocess
import time
from flask_ngrok import run_with_ngrok # Now this import should work

# This assumes your Flask app object is named 'app' in app.py
# If your app.py creates the app instance, then you can use run_with_ngrok directly on it.
# However, if you are running app.py as a subprocess, the flask_ngrok integration needs a slightly different approach.

# A simpler way to use run_with_ngrok if you're keeping your Flask app within the notebook:
# (If you're using %%writefile, the `app` object from app.py isn't directly available in the notebook's global scope after writing.)

# Option 1: If you want to use `run_with_ngrok` as intended (best practice if app is defined in notebook directly)
# (This would mean you copy the Flask app definition directly into this cell,
#  instead of writing it to app.py with %%writefile)
# from flask import Flask, render_template_string, request, jsonify
# from transformers import pipeline
# from flask_ngrok import run_with_ngrok

# app = Flask(__name__)
# sentiment_pipeline = pipeline("sentiment-analysis")
# # ... (your HTML_TEMPLATE, @app.route definitions etc.) ...
# if __name__ == '__main__':
#     run_with_ngrok(app) # Use run_with_ngrok directly with your app instance
#     app.run() # This line will actually be handled by run_with_ngrok

# Option 2: If you prefer to keep `app.py` separate and run it as a subprocess (more complex for ngrok integration)
# The original snippet for running as subprocess with ngrok.connect() is more suitable here.
# For `flask_ngrok`, it's designed to be used *within* the Flask app's main execution.

# Let's use `pyngrok` directly to expose the port, as `flask_ngrok` expects to control the app's run()
# This is generally more robust when running `app.py` as a separate process in Colab.

from pyngrok import ngrok

# Start Flask app in a separate thread/process
def run_flask_app():
    subprocess.run(["python", "app.py"])

flask_thread = threading.Thread(target=run_flask_app)
flask_thread.daemon = True
flask_thread.start()

# Give Flask some time to start up
time.sleep(5)

# Expose the Flask port (default 5000) using ngrok
public_url = ngrok.connect(5000)
print(f"Flask App Live at: {public_url}")
print("Please copy this URL and open in your browser.")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Flask App Live at: NgrokTunnel: "https://fa3da339a925.ngrok-free.app" -> "http://localhost:5000"
Please copy this URL and open in your browser.
